In [28]:
import pandas as pd
from sqlalchemy import create_engine
from config import password
password = password

In [9]:
# Define csvs
avo_data = "avo_prices.csv"
gas_data = "gas_prices.csv"

In [10]:
# Read in .csvs
avo_data_df = pd.read_csv(avo_data)
avo_data_df.head()

,Remaining Weeks in Year,Date,New Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,51,1/4/2015,1/5/2015,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
1,51,1/4/2015,1/5/2015,1.00,435021.49,364302.39,23821.16,82.15,46815.79,16707.15,30108.64,0.0,conventional,2015,Atlanta
2,51,1/4/2015,1/5/2015,1.08,788025.06,53987.31,552906.04,39995.03,141136.68,137146.07,3990.61,0.0,conventional,2015,BaltimoreWashington
3,51,1/4/2015,1/5/2015,1.01,80034.32,44562.12,24964.23,2752.35,7755.62,6064.30,1691.32,0.0,conventional,2015,Boise
4,51,1/4/2015,1/5/2015,1.02,491738.00,7193.87,396752.18,128.82,87663.13,87406.84,256.29,0.0,conventional,2015,Boston


In [11]:
# Read in .csvs
gas_data_df = pd.read_csv(gas_data)
gas_data_df.head()

,Date,Gasoline - All Grades,All Grades - Conventional Areas,All Grades - Reformulated Areas,Regular,Conventional Areas - Regular,Reformulated Areas - Regular,Midgrade,Conventional Areas - Midgrade,Reformulated Areas - Midgrade,Premium,Conventional Areas - Premium,Reformulated Areas - Premium,Diesel (On-Highway) - All Types,Ultra Low Sulfur (15 ppm and Under),Low Sulfer (Above 15 ppm to 500 ppm)
0,8/20/1990,NaN,NaN,NaN,1.191,1.191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8/27/1990,NaN,NaN,NaN,1.245,1.245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9/3/1990,NaN,NaN,NaN,1.242,1.242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9/10/1990,NaN,NaN,NaN,1.252,1.252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9/17/1990,NaN,NaN,NaN,1.266,1.266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Create a filtered dataframe from specific columns
avo_columns = ["New Date", "AveragePrice", "Total Volume", "4046", "4225", "4770", "type", "year", "region"]
avo_transformed_df = avo_data_df[avo_columns].copy()

# Rename the column headers
avo_transformed_df = avo_transformed_df.rename(columns={"New Date": "date",
                                                        "AveragePrice": "average_price",
                                                        "Total Volume": "total_volume",
                                                       "4046": "small_avocados_sold",
                                                       "4225": "large_avocados_sold",
                                                       "4770": "xl_avocados_sold",
                                                       "type": "type",
                                                       "year": "year",
                                                       "region": "region"})

avo_transformed_df.head()

,date,average_price,total_volume,small_avocados_sold,large_avocados_sold,xl_avocados_sold,type,year,region
0,1/5/2015,1.22,40873.28,2819.50,28287.42,49.90,conventional,2015,Albany
1,1/5/2015,1.00,435021.49,364302.39,23821.16,82.15,conventional,2015,Atlanta
2,1/5/2015,1.08,788025.06,53987.31,552906.04,39995.03,conventional,2015,BaltimoreWashington
3,1/5/2015,1.01,80034.32,44562.12,24964.23,2752.35,conventional,2015,Boise
4,1/5/2015,1.02,491738.00,7193.87,396752.18,128.82,conventional,2015,Boston


In [13]:
# Create a filtered dataframe from specific columns
gas_columns = ["Date", "Gasoline - All Grades", "Regular", "Midgrade", "Premium", "Diesel (On-Highway) - All Types"]
gas_transformed_df = gas_data_df[gas_columns].copy()

# Rename the column headers
gas_transformed_df = gas_transformed_df.rename(columns={"Date": "date",
                                                        "Gasoline - All Grades": "gas_all_grades",
                                                        "Regular": "regular",
                                                        "Midgrade": "midgrade",
                                                        "Premium": "premium",
                                                        "Diesel (On-Highway) - All Types": "diesel"})

gas_transformed_df.head()

,date,gas_all_grades,regular,midgrade,premium,diesel
0,8/20/1990,NaN,1.191,NaN,NaN,NaN
1,8/27/1990,NaN,1.245,NaN,NaN,NaN
2,9/3/1990,NaN,1.242,NaN,NaN,NaN
3,9/10/1990,NaN,1.252,NaN,NaN,NaN
4,9/17/1990,NaN,1.266,NaN,NaN,NaN


In [15]:
# Ensure dates have the same formatting
avo_transformed_df['date'] = pd.to_datetime(avo_transformed_df.date)
gas_transformed_df['date'] = pd.to_datetime(gas_transformed_df.date)

avo_transformed_df['date'] = avo_transformed_df['date'].dt.strftime('%m/%d/%Y')
gas_transformed_df['date'] = gas_transformed_df['date'].dt.strftime('%m/%d/%Y')

gas_transformed_df.head()

,date,gas_all_grades,regular,midgrade,premium,diesel
0,08/20/1990,NaN,1.191,NaN,NaN,NaN
1,08/27/1990,NaN,1.245,NaN,NaN,NaN
2,09/03/1990,NaN,1.242,NaN,NaN,NaN
3,09/10/1990,NaN,1.252,NaN,NaN,NaN
4,09/17/1990,NaN,1.266,NaN,NaN,NaN


In [16]:
# Filter data based on region and type
avo_transformed_df = avo_transformed_df.loc[avo_transformed_df['region'] == 'TotalUS']
avo_transformed_df = avo_transformed_df.loc[avo_transformed_df['type'] == 'conventional']
# from datetime import datetime, timedelta

# for date in avo_transformed_df['date']:
#     date += timedelta(days=1)
avo_transformed_df.head(20)

,date,average_price,total_volume,small_avocados_sold,large_avocados_sold,xl_avocados_sold,type,year,region
51,01/05/2015,0.95,31324277.73,12357161.34,13624083.05,844093.32,conventional,2015,TotalUS
159,01/12/2015,1.01,29063542.75,11544810.53,12134773.38,866574.66,conventional,2015,TotalUS
267,01/19/2015,1.03,29043458.85,11858139.34,11701947.80,831301.90,conventional,2015,TotalUS
375,01/26/2015,1.04,28470310.84,12167445.03,10734652.82,768020.05,conventional,2015,TotalUS
483,02/02/2015,0.89,44655461.51,18933038.04,18956479.74,1381516.11,conventional,2015,TotalUS
591,02/09/2015,0.95,32137333.01,13308193.40,13381347.54,737939.45,conventional,2015,TotalUS
699,02/16/2015,1.03,28012520.93,12626615.30,9783489.59,845653.52,conventional,2015,TotalUS
807,02/23/2015,1.02,29936729.76,12628562.36,11354281.64,937138.85,conventional,2015,TotalUS
915,03/02/2015,0.97,32994014.16,13282222.98,13733124.48,1070576.07,conventional,2015,TotalUS
1023,03/09/2015,1.02,30094698.85,13013750.35,10973972.60,834009.15,conventional,2015,TotalUS


In [22]:
# Create database connection
connection_string = (f"postgres:{password}@localhost:5432/etl_project_db")
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
avo_transformed_df.to_sql(name='avocado', con=engine, if_exists='append', index=False)
gas_transformed_df.to_sql(name='gas', con=engine, if_exists='append', index=False)

In [24]:
pd.read_sql_query('select * from avocado', con=engine).head()

,date,average_price,total_volume,small_avocados_sold,large_avocados_sold,xl_avocados_sold,type,year,region
0,01/05/2015,0.95,31324277.73,12357161.34,13624083.05,844093.32,conventional,2015,TotalUS
1,01/12/2015,1.01,29063542.75,11544810.53,12134773.38,866574.66,conventional,2015,TotalUS
2,01/19/2015,1.03,29043458.85,11858139.34,11701947.80,831301.90,conventional,2015,TotalUS
3,01/26/2015,1.04,28470310.84,12167445.03,10734652.82,768020.05,conventional,2015,TotalUS
4,02/02/2015,0.89,44655461.51,18933038.04,18956479.74,1381516.11,conventional,2015,TotalUS


In [25]:
pd.read_sql_query('select * from gas', con=engine).head()

,date,gas_all_grades,regular,midgrade,premium,diesel
0,08/20/1990,NaN,1.191,NaN,NaN,NaN
1,08/27/1990,NaN,1.245,NaN,NaN,NaN
2,09/03/1990,NaN,1.242,NaN,NaN,NaN
3,09/10/1990,NaN,1.252,NaN,NaN,NaN
4,09/17/1990,NaN,1.266,NaN,NaN,NaN
